# AOC2019 - Day 24

Today's challenge: [Day 24](https://adventofcode.com/2019/day/24)

Data are manually downloaded into `./data/input_24.txt`

In [1]:
with open('./data/input_24.txt') as f:
    raw = f.read()
    
rows = raw.split('\n')

In [2]:
from collections import defaultdict

from IPython import display
from time import sleep

In [3]:
class GameOfLife:
    def __init__(self, rows):
        self.panel = defaultdict(lambda: '.')
        for y, row in enumerate(rows):
            for x, c in enumerate(row):
                if c == '#':
                    self.panel[(x,y)] = '#'
                else:
                    self.panel[(x,y)] = '.'
        self.previous = set([str(self)])
        self.ds = [(1,0),(0,1),(-1,0),(0,-1)]
        
    def __repr__(self):
        res = ""
        for j in range(5):
            for i in range(5):
                res += self.panel[(i,j)]
            res += '\n'
        return res
    
    def score(self):
        res = 0
        for y in range(5):
            for x in range(5):
                if self.panel[(x,y)] == '#':
                    res += pow(2, 5*y+x)
        return res
    
    def nbs(self, x, y):
        return [(x+dx, y+dy) for dx, dy in self.ds]
    
    def count_sharp(self, x, y):
        return sum(self.panel[p] == '#' for p in self.nbs(x, y))
    
    def evolve(self):
        new_panel = defaultdict(lambda:'.')
        for i in range(5):
            for j in range(5):
                nbs = self.nbs(i,j)
                if self.panel[(i,j)] == '#':
                    new_panel[(i,j)] = '#' if self.count_sharp(i,j) == 1  else '.'
                if self.panel[(i,j)] == '.':
                    new_panel[(i,j)] = '#' if self.count_sharp(i,j) in [1,2] else '.'
        self.panel = new_panel
        if str(self) in self.previous:
            print(self)
            return False
        else:
            self.previous.add(str(self))
            return True

In [4]:
gol = GameOfLife(rows)
while gol.evolve():
    print(gol)
    display.clear_output(wait=True)
    sleep(0.1)
gol.score()

##.##
.###.
.#...
.#..#
##.##



28903899

In [5]:
class GameOfLifeRecursive:
    def __init__(self, rows):
        self.panel = defaultdict(lambda: '.')
        for y, row in enumerate(rows):
            for x, c in enumerate(row):
                if c == '#':
                    self.panel[(0,x,y)] = '#'
                else:
                    self.panel[(0,x,y)] = '.'
        self.min_layer = 0
        self.max_layer = 0
        self.ds = [(1,0),(0,1),(-1,0),(0,-1)]
        
    def __repr__(self):
        res = ""
        for l in range(self.min_layer, self.max_layer + 1):
            layer = "layer %s\n" % l
            for j in range(5):
                for i in range(5):
                    if (i, j) == (2,2):
                        layer += '?'
                    else:
                        layer += self.panel[(l,i,j)]
                layer += '\n'
            if '#' in layer:
                res += layer + "\n"
        return res
    
    def nbs(self, l, x, y):
        res = []
        for dx, dy in self.ds:
            if (x+dx, y+dy) == (2,2):
                res += [(l+1, *p) for p in self.side(dx,dy)]
            elif x + dx < 0:
                res += [(l-1, 1, 2)]
            elif x + dx > 4:
                res += [(l-1, 3, 2)]
            elif y + dy < 0:
                res += [(l-1, 2, 1)]
            elif y + dy > 4:
                res += [(l-1, 2, 3)]
            else:
                res.append((l, x+dx, y+dy))
        return res
    
    def count_sharp(self, l, x, y):
        return sum(self.panel[p] == '#' for p in self.nbs(l, x, y))
    
    @staticmethod
    def side(x,y):
        if x == 1:
            return [(0,i) for i in range(5)]
        elif x == -1:
            return [(4,i) for i in range(5)]
        elif y == 1:
            return [(i,0) for i in range(5)]
        elif y == -1:
            return [(i,4) for i in range(5)]
        else:
            print("WTF", x,y)
            
    def evolve(self):
        new_panel = defaultdict(lambda:'.')
        for l in range(self.min_layer-1, self.max_layer+2):
            for i in range(5):
                for j in range(5):
                    if (i, j) == (2,2):
                        continue
                    nbs = self.nbs(l,i,j)
                    if self.panel[(l,i,j)] == '#':
                        new_panel[(l,i,j)] = '#' if self.count_sharp(l,i,j) == 1  else '.'
                    if self.panel[(l,i,j)] == '.':
                        new_panel[(l,i,j)] = '#' if self.count_sharp(l,i,j) in [1,2] else '.'
        self.min_layer -= 1
        self.max_layer += 1
        self.panel = new_panel
        return self

In [6]:
gol = GameOfLifeRecursive(rows)
for _ in range(200):
    gol.evolve()

In [7]:
sum([1 for x in gol.panel.values() if x == '#'])

1896